***GENERATED CODE FOR trainingcompletedata PIPELINE.***

***DON'T EDIT THIS CODE.***

***CONNECTOR FUNCTIONS TO READ DATA.***

In [ ]:
import warnings
warnings.filterwarnings('ignore')


class RDBMSConnector:

    def fetch(spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        return spark.read.format("jdbc") \
            .option("url", f"jdbc:sqlserver://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option(eval(config)['qtype'], eval(config)['query']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .load()

    def put(df, spark, config):
        drivers = {"mssql": "com.microsoft.sqlserver.jdbc.SQLServerDriver"}
        # Write modes: overwrite, append
        df.write.mode(eval(config)['writemode'])\
            .format('jdbc') \
            .option("url", f"jdbc:{eval(config)['dbtype']}://{eval(config)['host']}:{eval(config)['port']};databaseName={eval(config)['database']}") \
            .option("dbtable", eval(config)['table']) \
            .option("user", eval(config)['user']) \
            .option("password", eval(config)['password']) \
            .option("driver",  "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
            .save()


***TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA***

In [ ]:
from pyspark.sql.functions import dayofmonth, month, year, col
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params, transformationData={}):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_stringindexer"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        outcol).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            outcol, dfReturn[outcol].cast(IntegerType()))
        return changed_type_df
    return dfReturn


def ExtractDateTransform(df, params, transformationData={}):
    transform_params = params
    dfReturn = df
    feature = transform_params['feature']
    dfReturn = dfReturn.fillna({feature: ''})
    dfReturn = dfReturn.withColumn(
        feature+'dayofmonth', dayofmonth(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'month', month(col(feature)))
    dfReturn = dfReturn.withColumn(feature+'year', year(col(feature)))
    return dfReturn


class TransformationMain:
    # TODO: change df argument in run with following
    def run(transformationDF, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'}], 'feature': 'ChargeDetailID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
                                                  'count': '500', 'mean': '', 'stddev': '', 'min': '001FCDEC-C753-4762-87D0-4E44D23242AC', 'max': 'FFF8B08A-2768-4BA1-A148-7A0CCEAC1DE9', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ChargeDetailID'}, {'feature_label': 'ChargeDetailID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ChargeDetailID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionID', 'transformation_label': 'String Indexer'}], 'feature': 'TransactionID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '00478280-BB8D-436E-8F24-9CA30EB68904', 'max': 'FF5D8944-D1B8-4A18-9CDC-D307D345BAC8', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TransactionID'}, {'feature_label': 'TransactionID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TransactionID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'PatientID', 'transformation_label': 'String Indexer'}], 'feature': 'PatientID', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': '00234AC9-DF2E-451B-81C6-892251D7A21B', 'max': 'FFD7F1B3-09BB-49C1-B45A-8921A5037828', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'PatientID'}, {'feature_label': 'PatientID', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('PatientID')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'}], 'feature': 'CPTCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '88261.6', 'stddev': '19971.38', 'min': '13132', 'max': 'J1580', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'CPTCode'}, {'feature_label': 'CPTCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('CPTCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'pdiagnosis', 'transformation_label': 'String Indexer'}], 'feature': 'pdiagnosis', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'B36.9', 'max': 'Z11.59', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'pdiagnosis'}, {'feature_label': 'pdiagnosis', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('pdiagnosis')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'cd_EntryDate', 'transformation_label': 'Extract Date'}], 'feature': 'cd_EntryDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'cd_EntryDate'}, {'feature_label': 'cd_EntryDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('cd_EntryDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 't_EntryDate', 'transformation_label': 'Extract Date'}], 'feature': 't_EntryDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 't_EntryDate'}, {'feature_label': 't_EntryDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('t_EntryDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 't_PostDate', 'transformation_label': 'Extract Date'}], 'feature': 't_PostDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 't_PostDate'}, {'feature_label': 't_PostDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('t_PostDate')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 't_CloseDate', 'transformation_label': 'Extract Date'}], 'feature': 't_CloseDate', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 't_CloseDate'}, {'feature_label': 't_CloseDate', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('t_CloseDate')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionDetailCode', 'transformation_label': 'String Indexer'}], 'feature': 'TransactionDetailCode', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'Adj Aetna', 'max': 'Pymt United Healthcare', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TransactionDetailCode'}, {'feature_label': 'TransactionDetailCode', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TransactionDetailCode')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'TransactionType', 'transformation_label': 'String Indexer'}], 'feature': 'TransactionType', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'A', 'max': 'C', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'TransactionType'}, {'feature_label': 'TransactionType', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('TransactionType')
        transformationDF = ExtractDateTransform(transformationDF, {'transformationsData': [{'feature_label': 'DateOfBirth', 'transformation_label': 'Extract Date'}], 'feature': 'DateOfBirth', 'type': 'date', 'selected': 'True', 'replaceby': 'random', 'stats': {
            'count': '', 'mean': '', 'stddev': '', 'min': '', 'max': '', 'missing': '0'}, 'transformation': [{'transformation': 'Extract Date', 'selectedAsDefault': 1}], 'generated': 'False', 'updatedLabel': 'DateOfBirth'}, {'feature_label': 'DateOfBirth', 'transformation_label': 'Extract Date'})
        transformationDF = transformationDF.drop('DateOfBirth')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'Gender', 'transformation_label': 'String Indexer'}], 'feature': 'Gender', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'F', 'max': 'M', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'Gender'}, {'feature_label': 'Gender', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('Gender')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'}], 'feature': 'DepartmentCategory', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'ALLERGY', 'max': 'SURGERY', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'DepartmentCategory'}, {'feature_label': 'DepartmentCategory', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('DepartmentCategory')
        transformationDF = StringIndexerTransform(transformationDF, {'transformationsData': [{'feature_label': 'ar', 'transformation_label': 'String Indexer'}], 'feature': 'ar', 'type': 'string', 'selected': 'True', 'replaceby': 'max', 'stats': {
            'count': '500', 'mean': '', 'stddev': '', 'min': 'AR120', 'max': 'AR90', 'missing': '0'}, 'transformation': [{'transformation': 'String Indexer', 'selectedAsDefault': 1}], 'updatedLabel': 'ar'}, {'feature_label': 'ar', 'transformation_label': 'String Indexer'})
        transformationDF = transformationDF.drop('ar')
        display(transformationDF.limit(2).toPandas())
        return transformationDF


***AUTOML FUNCTIONS***

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = (sparkDF.toPandas())
    X = (df.drop(label, axis=1))[listOfFeatures].values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=5,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))
    data = {'model': tpotModel,
            'X_test': X_test,
            'y_test': y_test,
            'label': label,
            'columnNames': listOfFeatures}
    return data


***READING DATAFRAME***

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

#%run trainingcompletedataHooks.ipynb
try:
	#sourcePreExecutionHook()

	sqlserver = RDBMSConnector.fetch(spark, "{'url': '/FileStore/platform/sampleData/5fb4bb43243223305178520a/part-00000-tid-3820678146467397328-c46d90f7-1f5e-4efd-b8c8-fd13984ec8c0-440-1-c000.csv', 'host': 'wsc-sqlserver1.database.windows.net', 'port': '1433', 'password': 'G5gTW5rnKVeAZWXm', 'database': 'Nextgen_Hent_Telehealth', 'optionalDB': 'Nextgen_Hent_Telehealth', 'user': 'Numtra', 'qtype': 'query', 'query': "SELECT DISTINCT  cd.ChargeDetailID,\r\nt.TransactionID,\r\ncd.PatientID,\r\ncd.Amount as charged_amount,\r\nt.Amount as paid_amount,\r\ncptc.CPTCode,\r\ndc.DiagnosisCode as pdiagnosis,\r\nDATEDIFF(day, cd.DateOfEntry, t.PostDate) as days_from_doe_to_first_post,\r\ncd.DateOfEntry as cd_EntryDate,\r\nt.DateOfEntry as t_EntryDate,\r\nt.PostDate as t_PostDate,\r\nt.CloseDate as t_CloseDate,\r\ntdc.TransactionDetailCode,\r\ntp.TransactionType,\r\np.DateOfBirth,\r\np.Gender,\r\nd.DepartmentCategory, \r\nar=(SELECT CASE\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=0 and DATEDIFF(day, cd.DateOfEntry, t.PostDate) <=30THEN'AR30'\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=31 and DATEDIFF(day, cd.DateOfEntry, t.PostDate) <=60THEN'AR60'\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=61 and DATEDIFF(day, cd.DateOfEntry, t.PostDate) <=90THEN'AR90'\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=91 and DATEDIFF(day, cd.DateOfEntry, t.PostDate) <=120THEN'AR120'\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=121 and DATEDIFF(day, cd.DateOfEntry, t.PostDate) <=150THEN'AR150'\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=151 and DATEDIFF(day, cd.DateOfEntry, t.PostDate) <=180THEN'AR180'\r\nWHEN DATEDIFF(day, cd.DateOfEntry, t.PostDate) >=181THEN'AR180Plus'\r\nEND),\r\nstatus=(SELECT CASE trc.ReasonType WHEN 'Denial' THEN 1 ELSE 0 END)\r\nFROM\r\n chargedetail as cd\r\ninner join Transactions t on cd.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionDetailCode as tdc on tdc.TransactionDetailCodeID = t.TransactionDetailCodeID\r\ninner join TransactionType tp on tp.TransactionTypeID = t.TransactionTypeID\r\ninner join TransactionReason tr on tr.TransactionID = t.TransactionID and tr.ChargeDetailID = t.ChargeDetailID\r\ninner join TransactionReasonCode trc on trc.TransactionReasonCodeID = tr.TransactionReasonCodeID\r\ninner join Patient p on p.PatientID = cd.PatientID\r\ninner join Department d on d.DepartmentID = cd.DepartmentID\r\ninner join Provider pp on pp.ProviderID = cd.ProviderID\r\ninner join Speciality s on s.SpecialityID = pp.SpecialityID\r\ninner join CptCode cptc on cptc.CPTCodeID = cd.CPTCodeID\r\ninner join DiagnosisCode dc on dc.DiagnosisCodeID = cd.DiagnosisCodeID1\r\nwhere\r\ncd.DateOfEntry between'1/1/2020'\r\nand'8/31/2020'\r\nand cd.Amount >.01", 'dbtype': 'mssql', 'is_header': 'Use Header Line'}")
	#sourcePostExecutionHook(sqlserver)

except Exception as ex: 
	logging.error(ex)


***TRANSFORMING DATAFRAME***

In [ ]:
#%run trainingcompletedataHooks.ipynb
try:
	#transformationPreExecutionHook()

	autofe = TransformationMain.run(sqlserver,json.dumps( {"FE": [{"transformationsData": [{"feature_label": "ChargeDetailID", "transformation_label": "String Indexer"}], "feature": "ChargeDetailID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "001FCDEC-C753-4762-87D0-4E44D23242AC", "max": "FFF8B08A-2768-4BA1-A148-7A0CCEAC1DE9", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ChargeDetailID"}, {"transformationsData": [{"feature_label": "TransactionID", "transformation_label": "String Indexer"}], "feature": "TransactionID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00478280-BB8D-436E-8F24-9CA30EB68904", "max": "FF5D8944-D1B8-4A18-9CDC-D307D345BAC8", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TransactionID"}, {"transformationsData": [{"feature_label": "PatientID", "transformation_label": "String Indexer"}], "feature": "PatientID", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "00234AC9-DF2E-451B-81C6-892251D7A21B", "max": "FFD7F1B3-09BB-49C1-B45A-8921A5037828", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "PatientID"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "charged_amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "345.33", "stddev": "745.86", "min": "40.0", "max": "12004.0", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "charged_amount"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "paid_amount", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "500", "mean": "119.7", "stddev": "510.52", "min": "-3590.82", "max": "9690.77", "missing": "0"}, "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "updatedLabel": "paid_amount"}, {"transformationsData": [{"feature_label": "CPTCode", "transformation_label": "String Indexer"}], "feature": "CPTCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "88261.6", "stddev": "19971.38", "min": "13132", "max": "J1580", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "CPTCode"}, {"transformationsData": [{"feature_label": "pdiagnosis", "transformation_label": "String Indexer"}], "feature": "pdiagnosis", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "B36.9", "max": "Z11.59", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "pdiagnosis"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "days_from_doe_to_first_post", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "18.83", "stddev": "18.5", "min": "3", "max": "143", "missing": "0"}, "updatedLabel": "days_from_doe_to_first_po..."}, {"transformationsData": [{"feature_label": "cd_EntryDate", "transformation_label": "Extract Date"}], "feature": "cd_EntryDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "cd_EntryDate"}, {"transformationsData": [{"feature_label": "t_EntryDate", "transformation_label": "Extract Date"}], "feature": "t_EntryDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "t_EntryDate"}, {"transformationsData": [{"feature_label": "t_PostDate", "transformation_label": "Extract Date"}], "feature": "t_PostDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "t_PostDate"}, {"transformationsData": [{"feature_label": "t_CloseDate", "transformation_label": "Extract Date"}], "feature": "t_CloseDate", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "t_CloseDate"}, {"transformationsData": [{"feature_label": "TransactionDetailCode", "transformation_label": "String Indexer"}], "feature": "TransactionDetailCode", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "Adj Aetna", "max": "Pymt United Healthcare", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TransactionDetailCode"}, {"transformationsData": [{"feature_label": "TransactionType", "transformation_label": "String Indexer"}], "feature": "TransactionType", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "A", "max": "C", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "TransactionType"}, {"transformationsData": [{"feature_label": "DateOfBirth", "transformation_label": "Extract Date"}], "feature": "DateOfBirth", "type": "date", "selected": "True", "replaceby": "random", "stats": {"count": "", "mean": "", "stddev": "", "min": "", "max": "", "missing": "0"}, "transformation": [{"transformation": "Extract Date", "selectedAsDefault": 1}], "generated": "False", "updatedLabel": "DateOfBirth"}, {"transformationsData": [{"feature_label": "Gender", "transformation_label": "String Indexer"}], "feature": "Gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "Gender"}, {"transformationsData": [{"feature_label": "DepartmentCategory", "transformation_label": "String Indexer"}], "feature": "DepartmentCategory", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "ALLERGY", "max": "SURGERY", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "DepartmentCategory"}, {"transformationsData": [{"feature_label": "ar", "transformation_label": "String Indexer"}], "feature": "ar", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "500", "mean": "", "stddev": "", "min": "AR120", "max": "AR90", "missing": "0"}, "transformation": [{"transformation": "String Indexer", "selectedAsDefault": 1}], "updatedLabel": "ar"}, {"transformationsData": [{"transformation_label": "novalue"}], "feature": "status", "transformation": [{"transformation": "novalue", "selectedAsDefault": 1}], "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "500", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "status"}, {"feature": "ChargeDetailID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "249.5", "stddev": "144.48", "min": "0.0", "max": "499.0", "missing": "0"}, "updatedLabel": "ChargeDetailID_stringinde..."}, {"feature": "TransactionID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "248.5", "stddev": "144.48", "min": "0.0", "max": "498.0", "missing": "0"}, "updatedLabel": "TransactionID_stringindex..."}, {"feature": "PatientID_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "239.7", "stddev": "144.15", "min": "0.0", "max": "489.0", "missing": "0"}, "updatedLabel": "PatientID_stringindexer_t..."}, {"feature": "CPTCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "7.47", "stddev": "10.88", "min": "0.0", "max": "58.0", "missing": "0"}, "updatedLabel": "CPTCode_stringindexer_tra..."}, {"feature": "pdiagnosis_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "24.13", "stddev": "31.48", "min": "0.0", "max": "125.0", "missing": "0"}, "updatedLabel": "pdiagnosis_stringindexer_..."}, {"feature": "cd_EntryDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "16.94", "stddev": "9.11", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "cd_EntryDate_dayofmonth"}, {"feature": "cd_EntryDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "4.24", "stddev": "2.45", "min": "1", "max": "8", "missing": "0"}, "updatedLabel": "cd_EntryDate_month"}, {"feature": "cd_EntryDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "cd_EntryDate_year"}, {"feature": "t_EntryDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.23", "stddev": "9.07", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "t_EntryDate_dayofmonth"}, {"feature": "t_EntryDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "4.88", "stddev": "2.49", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "t_EntryDate_month"}, {"feature": "t_EntryDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "t_EntryDate_year"}, {"feature": "t_PostDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.5", "stddev": "8.96", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "t_PostDate_dayofmonth"}, {"feature": "t_PostDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "4.9", "stddev": "2.48", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "t_PostDate_month"}, {"feature": "t_PostDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "t_PostDate_year"}, {"feature": "t_CloseDate_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "16.9", "stddev": "8.97", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "t_CloseDate_dayofmonth"}, {"feature": "t_CloseDate_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "4.85", "stddev": "2.5", "min": "1", "max": "10", "missing": "0"}, "updatedLabel": "t_CloseDate_month"}, {"feature": "t_CloseDate_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "2020.0", "stddev": "0.0", "min": "2020", "max": "2020", "missing": "0"}, "updatedLabel": "t_CloseDate_year"}, {"feature": "TransactionDetailCode_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "5.47", "stddev": "4.91", "min": "0.0", "max": "19.0", "missing": "0"}, "updatedLabel": "TransactionDetailCode_str..."}, {"feature": "TransactionType_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.47", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "TransactionType_stringind..."}, {"feature": "DateOfBirth_dayofmonth", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "15.65", "stddev": "8.55", "min": "1", "max": "31", "missing": "0"}, "updatedLabel": "DateOfBirth_dayofmonth"}, {"feature": "DateOfBirth_month", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "6.23", "stddev": "3.35", "min": "1", "max": "12", "missing": "0"}, "updatedLabel": "DateOfBirth_month"}, {"feature": "DateOfBirth_year", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "generated": "True", "selected": "True", "stats": {"count": "500", "mean": "1966.13", "stddev": "21.37", "min": "1925", "max": "2017", "missing": "0"}, "updatedLabel": "DateOfBirth_year"}, {"feature": "Gender_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "numeric", "selected": "True", "stats": {"count": "500", "mean": "0.44", "stddev": "0.5", "min": "0", "max": "1", "missing": "0"}, "updatedLabel": "Gender_stringindexer_tran..."}, {"feature": "DepartmentCategory_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "1.42", "stddev": "1.56", "min": "0.0", "max": "9.0", "missing": "0"}, "updatedLabel": "DepartmentCategory_string..."}, {"feature": "ar_stringindexer_transform", "transformation": [{"transformation": "novalue", "selectedAsDefault": 0}], "transformationsData": [{"transformation_label": "novalue"}], "type": "real", "selected": "True", "stats": {"count": "500", "mean": "0.17", "stddev": "0.56", "min": "0.0", "max": "4.0", "missing": "0"}, "updatedLabel": "ar_stringindexer_transfor..."}]}))

	#transformationPostExecutionHook(autofe)

except Exception as ex: 
	logging.error(ex)


***TRAIN MODEL***

In [ ]:
#%run trainingcompletedataHooks.ipynb
try:
	#mlPreExecutionHook()

	dataAutoML=functionClassification(autofe, ["ChargeDetailID_stringindexer", "TransactionID_stringindexer", "PatientID_stringindexer", "charged_amount", "paid_amount", "CPTCode_stringindexer", "pdiagnosis_stringindexer", "days_from_doe_to_first_post", "TransactionDetailCode_stringindexer", "TransactionType_stringindexer", "Gender_stringindexer", "DepartmentCategory_stringindexer", "ar_stringindexer"], "status")

	#mlPostExecutionHook(dataAutoML)

except Exception as ex: 
	logging.error(ex)


***PREDICT ON TRAINED MODEL***

In [ ]:
import pandas as pd
import numpy as np
import sklearn.metrics

try:
    model=dataAutoML['model']
    X_test=dataAutoML['X_test']
    y_test=dataAutoML['y_test']
    label=dataAutoML['label']
    columnNames=dataAutoML['columnNames']
    if label in columnNames:
        columnNames.remove(label)
    predicted=label+"_predicted"
    y_predicted=model.predict(X_test)
    df =pd.DataFrame(X_test , columns=columnNames)
    df[label]=y_test
    df[predicted]=y_predicted
    columnNames.insert(0,predicted)
    columnNames.insert(0,label)
    Accuracy = np.round((100 * sklearn.metrics.accuracy_score(y_true=y_test, y_pred=y_predicted)), 1)
    F1= np.round(
            (100 * sklearn.metrics.f1_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Precision= np.round((
                100 * sklearn.metrics.precision_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    Recall = np.round((
                100 * sklearn.metrics.recall_score(y_true=y_test, y_pred=y_predicted, average="weighted")), 1)
    display(" Accuracy of Prediction on test data    : %s"%Accuracy)
    display(" F1 score of Prediction on test data    : %s"%F1)
    display(" Precision of Prediction on test data   : %s"%Precision)
    display(" Recall of Prediction on test data      : %s"%Recall)
    display(df.head())
except Exception as ex:
    logging.error(ex)

